# Lab 10: Multi-Layer Perceptrons (MLP) & Backpropagation

In this lab, you'll build intuition for how neural networks learn by exploring MLPs on non-linearly separable data. You'll visualize decision boundaries, experiment with activation functions, and tune hyperparameters.

### Learning Objectives

- Understand the architecture of multi-layer perceptrons
- Visualize how hidden layers enable non-linear decision boundaries
- Compare activation functions (sigmoid, tanh, ReLU)
- Explore hyperparameter effects (learning rate, hidden units, epochs)
- Use train/validation/test splits for model selection

### Overview

| Part | Topic                                |
| ---- | ------------------------------------ |
| 1    | MLP Basics & Non-Linear Data         |
| 2    | Activation Functions                 |
| 3    | Network Architecture (Hidden Layers) |
| 4    | Hyperparameter Tuning                |


---

## Setup


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    ConfusionMatrixDisplay,
)

import warnings

warnings.filterwarnings(
    "ignore", category=UserWarning
)  # Suppress convergence warnings for cleaner output

# Set random seed for reproducibility
np.random.seed(42)

### Helper Functions


In [ ]:
def plot_decision_boundary(clf, X, y, ax=None, title="Decision Boundary", cmap="RdBu"):
    """
    Plot the decision boundary of a classifier.

    Parameters:
    -----------
    clf : fitted classifier
    X : feature array (n_samples, 2)
    y : target array
    ax : matplotlib axis (optional)
    title : plot title
    cmap : colormap for decision regions
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))

    # Create mesh grid
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))

    # Predict on mesh
    if hasattr(clf, "predict_proba"):
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    else:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot decision regions
    ax.contourf(xx, yy, Z, levels=50, cmap=cmap, alpha=0.4)
    ax.contour(xx, yy, Z, levels=[0.5], colors="k", linewidths=2)

    # Plot data points
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap, edgecolors="k", s=50)

    ax.set_xlabel("Feature 1")
    ax.set_ylabel("Feature 2")
    ax.set_title(title)

    return ax


def plot_loss_curve(clf, ax=None, title="Loss Curve"):
    """
    Plot the training loss curve of an MLP classifier.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 5))

    ax.plot(clf.loss_curve_, linewidth=2)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Loss")
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

    return ax

---

## Part 1: MLP Basics & Non-Linear Data

### The Multi-Layer Perceptron

An MLP consists of:

- **Input layer:** Receives the features
- **Hidden layer(s):** Transform the input through learned weights and non-linear activation functions
- **Output layer:** Produces the prediction

The key insight is that hidden layers with non-linear activations allow the network to learn **non-linear decision boundaries** — something a single perceptron (or linear classifier) cannot do.

### Backpropagation

The network learns by:

1. **Forward pass:** Compute predictions
2. **Compute loss:** Measure error between predictions and true labels
3. **Backward pass:** Propagate gradients back through the network
4. **Update weights:** Adjust weights to reduce loss

This is repeated for many **epochs** (passes through the training data).


### 1.1 Non-Linearly Separable Data

Let's create datasets that **cannot** be separated by a straight line. This is where MLPs shine compared to linear models.


In [ ]:
# Generate non-linearly separable datasets
X_moons, y_moons = make_moons(n_samples=500, noise=0.3, random_state=42)
X_circles, y_circles = make_circles(
    n_samples=500, noise=0.2, factor=0.5, random_state=42
)

# Visualize the manifold structure
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].scatter(
    X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap="RdBu", edgecolors="k", s=40
)
axes[0].set_title("Moons Dataset\n(Two interleaving crescents)", fontsize=12)
axes[0].set_xlabel("Feature 1")
axes[0].set_ylabel("Feature 2")

axes[1].scatter(
    X_circles[:, 0], X_circles[:, 1], c=y_circles, cmap="RdBu", edgecolors="k", s=40
)
axes[1].set_title("Circles Dataset\n(Concentric rings)", fontsize=12)
axes[1].set_xlabel("Feature 1")
axes[1].set_ylabel("Feature 2")

plt.suptitle("Non-Linearly Separable Data: The Manifold Structure", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("Notice how no straight line can separate the two classes in either dataset.")
print(
    "These datasets have complex 'manifold structure' that requires non-linear boundaries."
)

### 1.2 Data Preparation: Train / Validation / Test Split

For proper model development, we split our data into **three** partitions:

- **Training set (60%):** Used to train the model
- **Validation set (20%):** Used to tune hyperparameters and select the best model
- **Test set (20%):** Used only once at the end to estimate real-world performance

This prevents us from overfitting to the test set during hyperparameter tuning.


In [ ]:
# We'll use the moons dataset for most of this lab
X = X_moons
y = y_moons

# First split: separate out test set (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Second split: separate train and validation from the remaining 80%
X_train, X_val, y_train, y_val = train_test_split(
    X_temp,
    y_temp,
    test_size=0.25,
    random_state=42,
    stratify=y_temp,  # 0.25 of 80% = 20% overall
)

print(f"Training set:   {len(X_train)} samples ({len(X_train) / len(X) * 100:.0f}%)")
print(f"Validation set: {len(X_val)} samples ({len(X_val) / len(X) * 100:.0f}%)")
print(f"Test set:       {len(X_test)} samples ({len(X_test) / len(X) * 100:.0f}%)")

### 1.3 Scale the Features

Neural networks are sensitive to feature scales. We standardize so that each feature has mean ≈ 0 and std ≈ 1.

**Important:** We fit the scaler on the training data only, then transform all sets.


In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit on training data only
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(
    f"Training data - mean: {X_train_scaled.mean(axis=0).round(3)}, std: {X_train_scaled.std(axis=0).round(3)}"
)

### 1.4 Train Your First MLP

Let's train a simple MLP with one hidden layer of 20 neurons.


In [ ]:
# Create and train an MLP
mlp = MLPClassifier(
    hidden_layer_sizes=(20,),  # One hidden layer with 20 neurons
    activation="relu",  # ReLU activation function
    solver="adam",  # Adam optimizer
    max_iter=1000,  # Maximum epochs
    random_state=0,  # Different seed for better demo
)

mlp.fit(X_train_scaled, y_train)

# Evaluate
train_acc = mlp.score(X_train_scaled, y_train)
val_acc = mlp.score(X_val_scaled, y_val)

print(f"Training accuracy:   {train_acc:.3f}")
print(f"Validation accuracy: {val_acc:.3f}")
print(f"\nNumber of iterations (epochs): {mlp.n_iter_}")

In [ ]:
# Visualize the decision boundary on both training and validation data
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Decision boundary - Training data (what the model learned from)
plot_decision_boundary(
    mlp,
    X_train_scaled,
    y_train,
    ax=axes[0],
    title=f"Training Data\nAccuracy: {train_acc:.3f}",
)

# Decision boundary - Validation data (unseen during training)
plot_decision_boundary(
    mlp,
    X_val_scaled,
    y_val,
    ax=axes[1],
    title=f"Validation Data\nAccuracy: {val_acc:.3f}",
)

# Loss curve
plot_loss_curve(mlp, ax=axes[2], title="Training Loss Over Epochs")

plt.tight_layout()
plt.show()

print("Key observation: The MLP has learned a NON-LINEAR decision boundary!")
print(
    "Compare: Training data (left) vs Validation data (middle) - same boundary, different points."
)

### 1.5 Discussion

**Observe:**

1. The MLP has learned a **non-linear decision boundary** that curves around the data
2. The loss decreases over training iterations (epochs)
3. Compare train vs. validation accuracy — if train >> validation, the model may be overfitting

**Questions to consider:**

- What would happen with more hidden neurons?
- What if we trained for more epochs?
- What if we used a different activation function?


---

## Part 2: Activation Functions

The **activation function** introduces non-linearity into the network. Without it, stacking layers would be equivalent to a single linear transformation.

### Common Activation Functions

| Function    | Formula                                        | Range   | Notes                                               |
| ----------- | ---------------------------------------------- | ------- | --------------------------------------------------- |
| **Sigmoid** | $\sigma(x) = \frac{1}{1 + e^{-x}}$             | (0, 1)  | Historical; suffers from vanishing gradients        |
| **Tanh**    | $\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$ | (-1, 1) | Zero-centered; still has vanishing gradient issue   |
| **ReLU**    | $\text{ReLU}(x) = \max(0, x)$                  | [0, ∞)  | Most common today; fast, avoids vanishing gradients |


### 2.1 Visualize Activation Functions


In [ ]:
# Define activation functions
x = np.linspace(-5, 5, 200)

sigmoid = 1 / (1 + np.exp(-x))
tanh = np.tanh(x)
relu = np.maximum(0, x)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(x, sigmoid, "b-", linewidth=2)
axes[0].axhline(y=0, color="k", linewidth=0.5)
axes[0].axvline(x=0, color="k", linewidth=0.5)
axes[0].set_title("Sigmoid\n$\\sigma(x) = 1/(1+e^{-x})$", fontsize=12)
axes[0].set_xlabel("x")
axes[0].set_ylabel("σ(x)")
axes[0].set_ylim(-0.2, 1.2)
axes[0].grid(True, alpha=0.3)

axes[1].plot(x, tanh, "g-", linewidth=2)
axes[1].axhline(y=0, color="k", linewidth=0.5)
axes[1].axvline(x=0, color="k", linewidth=0.5)
axes[1].set_title("Tanh\n$\\tanh(x) = (e^x - e^{-x})/(e^x + e^{-x})$", fontsize=12)
axes[1].set_xlabel("x")
axes[1].set_ylabel("tanh(x)")
axes[1].set_ylim(-1.2, 1.2)
axes[1].grid(True, alpha=0.3)

axes[2].plot(x, relu, "r-", linewidth=2)
axes[2].axhline(y=0, color="k", linewidth=0.5)
axes[2].axvline(x=0, color="k", linewidth=0.5)
axes[2].set_title("ReLU\n$\\text{ReLU}(x) = \\max(0, x)$", fontsize=12)
axes[2].set_xlabel("x")
axes[2].set_ylabel("ReLU(x)")
axes[2].set_ylim(-0.5, 5.5)
axes[2].grid(True, alpha=0.3)

plt.suptitle("Activation Functions", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 2.2 Gradients and the Vanishing Gradient Problem

During backpropagation, gradients are multiplied through each layer. The **derivative** of the activation function determines how gradients flow backward.

Look at the derivatives:

- **Sigmoid:** $\sigma'(x) = \sigma(x)(1 - \sigma(x))$ → max value is 0.25 at x=0
- **Tanh:** $\tanh'(x) = 1 - \tanh^2(x)$ → max value is 1 at x=0
- **ReLU:** $\text{ReLU}'(x) = 1$ if $x > 0$, else $0$ → gradient is 1 or 0

**The problem:** When inputs are far from zero, sigmoid and tanh derivatives become very small (near 0). Multiplying many small numbers through layers causes gradients to **vanish**, making early layers learn very slowly.

**Why ReLU wins:** The gradient is either 0 or 1 — no shrinking. This allows gradients to flow through deep networks.


In [ ]:
# Compute derivatives of activation functions
x = np.linspace(-5, 5, 200)

# Activation functions
sigmoid = 1 / (1 + np.exp(-x))
tanh_vals = np.tanh(x)

# Derivatives
sigmoid_deriv = sigmoid * (1 - sigmoid)
tanh_deriv = 1 - tanh_vals**2
relu_deriv = (x > 0).astype(float)

# Plot derivatives
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(x, sigmoid_deriv, "b-", linewidth=2)
axes[0].axhline(y=0.25, color="r", linestyle="--", alpha=0.7, label="max = 0.25")
axes[0].axhline(y=0, color="k", linewidth=0.5)
axes[0].axvline(x=0, color="k", linewidth=0.5)
axes[0].fill_between(x, sigmoid_deriv, alpha=0.3)
axes[0].set_title(
    "Sigmoid Derivative\n$\\sigma'(x) = \\sigma(x)(1-\\sigma(x))$", fontsize=12
)
axes[0].set_xlabel("x")
axes[0].set_ylabel("σ'(x)")
axes[0].set_ylim(-0.05, 0.35)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(x, tanh_deriv, "g-", linewidth=2)
axes[1].axhline(y=1, color="r", linestyle="--", alpha=0.7, label="max = 1")
axes[1].axhline(y=0, color="k", linewidth=0.5)
axes[1].axvline(x=0, color="k", linewidth=0.5)
axes[1].fill_between(x, tanh_deriv, alpha=0.3, color="green")
axes[1].set_title("Tanh Derivative\n$\\tanh'(x) = 1 - \\tanh^2(x)$", fontsize=12)
axes[1].set_xlabel("x")
axes[1].set_ylabel("tanh'(x)")
axes[1].set_ylim(-0.1, 1.2)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(x, relu_deriv, "r-", linewidth=2)
axes[2].axhline(y=1, color="r", linestyle="--", alpha=0.7, label="gradient = 1")
axes[2].axhline(y=0, color="k", linewidth=0.5)
axes[2].axvline(x=0, color="k", linewidth=0.5)
axes[2].fill_between(x, relu_deriv, alpha=0.3, color="red")
axes[2].set_title("ReLU Derivative\n$\\text{ReLU}'(x) = 1$ if $x>0$", fontsize=12)
axes[2].set_xlabel("x")
axes[2].set_ylabel("ReLU'(x)")
axes[2].set_ylim(-0.1, 1.3)
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.suptitle("Activation Function Derivatives (Gradients)", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("Key insight: For |x| > 2, sigmoid and tanh derivatives are nearly 0.")
print("In deep networks, multiplying many small gradients → vanishing gradients.")
print("ReLU maintains gradient = 1 for positive inputs, enabling deeper networks.")

### 2.3 Compare Activation Functions on the Moons Dataset

Let's train MLPs with different activation functions and compare their decision boundaries.


In [ ]:
activations = ["logistic", "tanh", "relu"]  # 'logistic' is sklearn's name for sigmoid
activation_names = ["Sigmoid", "Tanh", "ReLU"]

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for ax, activation, name in zip(axes, activations, activation_names):
    # Train MLP with this activation
    mlp = MLPClassifier(
        hidden_layer_sizes=(12,),
        activation=activation,
        solver="adam",
        max_iter=500,
        random_state=42,
    )
    mlp.fit(X_train_scaled, y_train)

    # Get accuracies
    train_acc = mlp.score(X_train_scaled, y_train)
    val_acc = mlp.score(X_val_scaled, y_val)

    # Plot decision boundary
    plot_decision_boundary(
        mlp,
        X_train_scaled,
        y_train,
        ax=ax,
        title=f"{name}\nTrain: {train_acc:.3f}, Val: {val_acc:.3f}",
    )

plt.suptitle(
    "Decision Boundaries with Different Activation Functions", fontsize=14, y=1.02
)
plt.tight_layout()
plt.show()

### 2.4 Compare Loss Curves

Different activations can lead to different training dynamics. Let's compare how quickly each converges.


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors = ["blue", "green", "red"]

for activation, name, color in zip(activations, activation_names, colors):
    mlp = MLPClassifier(
        hidden_layer_sizes=(10,),
        activation=activation,
        solver="adam",
        max_iter=1000,
        n_iter_no_change=1000,  # Disable early stopping to train full 2000 epochs
        random_state=42,
    )
    mlp.fit(X_train_scaled, y_train)

    ax.plot(mlp.loss_curve_, label=name, color=color, linewidth=2)

ax.set_xlabel("Iteration (Epoch)", fontsize=12)
ax.set_ylabel("Loss", fontsize=12)
ax.set_title("Training Loss Curves by Activation Function", fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Observations:")
print(
    "• ReLU (red) achieves the lowest final loss — gradients flow well, enabling continued learning"
)
print(
    "• Sigmoid (blue) plateaus at a higher loss — this is the vanishing gradient problem!"
)
print(
    "• Tanh (green) is in between — better than sigmoid but still saturates for large inputs"
)
print(
    "\nThis demonstrates WHY ReLU became the default activation for modern neural networks."
)

### 2.5 Your Turn: Test on Circles Dataset

**Task:** Train MLPs with different activation functions on the circles dataset. Which activation works best?


In [ ]:
# Split the circles dataset (same way we split moons)
X_temp_c, X_test_c, y_temp_c, y_test_c = train_test_split(
    X_circles, y_circles, test_size=0.2, random_state=42, stratify=y_circles
)
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(
    X_temp_c, y_temp_c, test_size=0.25, random_state=42, stratify=y_temp_c
)

# Scale the features
scaler_c = StandardScaler()
X_train_c_scaled = scaler_c.fit_transform(X_train_c)
X_val_c_scaled = scaler_c.transform(X_val_c)

print(
    f"Training: {len(X_train_c)} | Validation: {len(X_val_c)} | Test: {len(X_test_c)}\n"
)

# TODO: Train MLPs with each activation function and compare
# Use the lists below to loop through activations
activations = ["logistic", "tanh", "relu"]
activation_names = ["Sigmoid", "Tanh", "ReLU"]

# TODO: Create a figure with 3 subplots (like section 2.3)
# fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# TODO: Loop through activations, train an MLP, and plot decision boundaries
# Hint: Use plot_decision_boundary(mlp, X_train_c_scaled, y_train_c, ax=ax, title=...)

---

## Part 3: Network Architecture (Hidden Layers)

The **hidden layer sizes** determine the network's capacity to learn complex patterns:

- **More neurons (width):** Can learn more complex boundaries, but risk overfitting
- **More layers (depth):** Can learn hierarchical representations, but harder to train

In sklearn's `MLPClassifier`, `hidden_layer_sizes=(50, 30)` means:

- First hidden layer: 50 neurons
- Second hidden layer: 30 neurons


### 3.1 Effect of Number of Hidden Neurons

Let's see how the decision boundary changes with different numbers of neurons in a single hidden layer.


In [ ]:
hidden_sizes = [1, 2, 5, 10, 50, 100]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, n_hidden in zip(axes, hidden_sizes):
    mlp = MLPClassifier(
        hidden_layer_sizes=(n_hidden,),
        activation="relu",
        solver="adam",
        max_iter=500,
        random_state=42,
    )
    mlp.fit(X_train_scaled, y_train)

    train_acc = mlp.score(X_train_scaled, y_train)
    val_acc = mlp.score(X_val_scaled, y_val)

    plot_decision_boundary(
        mlp,
        X_train_scaled,
        y_train,
        ax=ax,
        title=f"{n_hidden} Hidden Neurons\nTrain: {train_acc:.3f}, Val: {val_acc:.3f}",
    )

plt.suptitle("Effect of Number of Hidden Neurons (Single Layer)", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

**Discussion:**

1. With very few neurons (1-2), the network cannot capture the non-linear structure
2. With more neurons (5-10), the boundary becomes flexible enough
3. With many neurons (50-100), the boundary may become too flexible (overfitting)


### 3.2 Effect of Number of Hidden Layers (Depth)

A network with **2 or more hidden layers** is often called a **deep neural network** — this is where "deep learning" gets its name!

**Why does depth matter?**

- Each layer can learn increasingly **abstract representations** of the input
- Deeper networks can compose simpler features into complex patterns
- In practice, deep networks excel at hierarchical data (images, text, audio)

However, for simple 2D datasets like ours, depth may not provide much benefit. Let's experiment by comparing architectures with **similar parameter counts** (~150 parameters each) but different depths.


In [ ]:
# Different architectures with SIMILAR parameter counts (~150 params each)
# Formula: params = sum of (inputs × outputs + biases) for each layer connection
architectures = [
    (37,),  # 1 layer: (2×37+37) + (37×1+1) = 149 params
    (10, 10),  # 2 layers: (2×10+10) + (10×10+10) + (10×1+1) = 151 params
    (7, 7, 7),  # 3 layers: (2×7+7) + 2×(7×7+7) + (7×1+1) = 141 params
    (6, 6, 6, 6),  # 4 layers: (2×6+6) + 3×(6×6+6) + (6×1+1) = 151 params
]


def count_params(input_dim, hidden_sizes, output_dim=1):
    """Count total parameters in an MLP."""
    layers = [input_dim] + list(hidden_sizes) + [output_dim]
    total = 0
    for i in range(len(layers) - 1):
        total += layers[i] * layers[i + 1] + layers[i + 1]  # weights + biases
    return total


fig, axes = plt.subplots(1, 4, figsize=(18, 4))

for ax, arch in zip(axes, architectures):
    mlp = MLPClassifier(
        hidden_layer_sizes=arch,
        activation="relu",
        solver="adam",
        max_iter=500,
        random_state=42,
    )
    mlp.fit(X_train_scaled, y_train)

    train_acc = mlp.score(X_train_scaled, y_train)
    val_acc = mlp.score(X_val_scaled, y_val)

    # Count parameters
    n_params = count_params(2, arch, 1)
    arch_str = " → ".join(map(str, arch))
    n_layers = len(arch)

    plot_decision_boundary(
        mlp,
        X_train_scaled,
        y_train,
        ax=ax,
        title=f"{n_layers} Layer(s): {arch_str}\n{n_params} params | Train: {train_acc:.3f}, Val: {val_acc:.3f}",
    )

plt.suptitle("Effect of Network Depth (Similar Parameter Count)", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print(
    "All architectures have ~150 parameters, so differences are due to DEPTH, not capacity."
)
print("Observation: For this simple 2D problem, shallow networks work just as well.")
print("Deeper networks shine on more complex, hierarchical data (e.g., images).")

### 3.3 Visualize Network Architecture

Let's visualize what a simple MLP architecture looks like.


In [ ]:
def draw_neural_network(ax, layer_sizes, title="Neural Network Architecture"):
    """
    Draw a simple neural network diagram.
    """
    n_layers = len(layer_sizes)
    max_neurons = max(layer_sizes)

    # Positions
    layer_positions = np.linspace(0, 1, n_layers)

    for i, (n_neurons, x_pos) in enumerate(zip(layer_sizes, layer_positions)):
        # Vertical positions for neurons in this layer
        y_positions = np.linspace(0.2, 0.8, n_neurons) if n_neurons > 1 else [0.5]

        for y_pos in y_positions:
            # Draw neuron
            circle = plt.Circle(
                (x_pos, y_pos), 0.03, color="steelblue", ec="black", zorder=4
            )
            ax.add_patch(circle)

            # Draw connections to next layer
            if i < n_layers - 1:
                next_n_neurons = layer_sizes[i + 1]
                next_y_positions = (
                    np.linspace(0.2, 0.8, next_n_neurons)
                    if next_n_neurons > 1
                    else [0.5]
                )
                next_x = layer_positions[i + 1]

                for next_y in next_y_positions:
                    ax.plot(
                        [x_pos + 0.03, next_x - 0.03],
                        [y_pos, next_y],
                        "gray",
                        alpha=0.3,
                        linewidth=0.5,
                        zorder=1,
                    )

    # Labels
    labels = ["Input"] + [f"Hidden {i + 1}" for i in range(n_layers - 2)] + ["Output"]
    for x_pos, label, n in zip(layer_positions, labels, layer_sizes):
        ax.text(x_pos, 0.05, f"{label}\n({n})", ha="center", fontsize=10)

    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(0, 1)
    ax.set_aspect("equal")
    ax.axis("off")
    ax.set_title(title, fontsize=12)


# Draw example architectures
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

draw_neural_network(axes[0], [2, 4, 1], "Simple: 2 → 4 → 1")
draw_neural_network(axes[1], [2, 6, 4, 1], "Medium: 2 → 6 → 4 → 1")
draw_neural_network(axes[2], [2, 8, 6, 4, 1], "Deep: 2 → 8 → 6 → 4 → 1")

plt.suptitle("MLP Architectures", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 3.4 Your Turn: Find a Good Architecture

**Task:** Experiment with different architectures. Try to find one that achieves high validation accuracy without overfitting.


In [ ]:
# TODO: Try different hidden_layer_sizes
# Examples: (5,), (10, 5), (20, 10, 5), etc.

# TODO: Compare training and validation accuracy
# A big gap suggests overfitting

---

## Part 4: Hyperparameter Tuning

Key hyperparameters for MLPs:

- **Learning rate (`learning_rate_init`):** Step size for weight updates
- **Hidden layer sizes:** Network architecture
- **Number of epochs (`max_iter`):** How long to train
- **Regularization (`alpha`):** L2 penalty to prevent overfitting

### The Learning Rate

- **Too high:** Training is unstable, loss oscillates or diverges
- **Too low:** Training is slow, may get stuck in local minima
- **Just right:** Smooth convergence to a good solution


### 4.1 Effect of Learning Rate


In [ ]:
learning_rates = [0.0001, 0.001, 0.01, 0.1, 1]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot loss curves
for lr in learning_rates:
    mlp = MLPClassifier(
        hidden_layer_sizes=(10,),
        activation="relu",
        solver="sgd",  # Use SGD to see learning rate effects more clearly
        learning_rate_init=lr,
        max_iter=400,
        n_iter_no_change=400,  # Disable early stopping to train full 2000 epochs
        random_state=42,
    )
    mlp.fit(X_train_scaled, y_train)
    axes[0].plot(mlp.loss_curve_, label=f"lr={lr}", linewidth=2)

axes[0].set_xlabel("Iteration")
axes[0].set_ylabel("Loss")
axes[0].set_title("Loss Curves for Different Learning Rates")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot validation accuracy vs learning rate
lr_range = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 1]
val_accs = []

for lr in lr_range:
    mlp = MLPClassifier(
        hidden_layer_sizes=(10,),
        activation="relu",
        solver="sgd",
        learning_rate_init=lr,
        max_iter=400,
        random_state=42,
    )
    mlp.fit(X_train_scaled, y_train)
    val_accs.append(mlp.score(X_val_scaled, y_val))

axes[1].semilogx(lr_range, val_accs, "bo-", linewidth=2, markersize=8)
axes[1].set_xlabel("Learning Rate (log scale)")
axes[1].set_ylabel("Validation Accuracy")
axes[1].set_title("Validation Accuracy vs Learning Rate")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observations:")
print("• Too LOW (0.0001): Loss barely decreases — learning is too slow")
print("• Too HIGH (1.0): Loss oscillates wildly — updates overshoot the minimum")
print("• JUST RIGHT (0.01-0.1): Smooth, fast convergence to low loss")
print(
    "\nThe right plot shows accuracy drops at very high learning rates — the model becomes unstable."
)

### 4.2 Effect of Number of Epochs (Overfitting)

Training for too many epochs can lead to **overfitting** — the model memorizes the training data instead of learning general patterns.


In [ ]:
# More granular view of training vs validation accuracy
max_iters = list(range(10, 1000, 20))  # Every 20 epochs from 10 to 1000

train_accs = []
val_accs = []

for max_iter in max_iters:
    mlp = MLPClassifier(
        hidden_layer_sizes=(1000,),  # More capacity to overfit
        activation="relu",
        solver="adam",
        max_iter=max_iter,
        random_state=0,
    )
    mlp.fit(X_train_scaled, y_train)
    train_accs.append(mlp.score(X_train_scaled, y_train))
    val_accs.append(mlp.score(X_val_scaled, y_val))

# Plot
plt.figure(figsize=(10, 6))
plt.plot(max_iters, train_accs, "b-", label="Training", linewidth=2)
plt.plot(max_iters, val_accs, "r-", label="Validation", linewidth=2)
plt.xlabel("Number of Epochs (max_iter)", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)
plt.title("Training vs Validation Accuracy by Number of Epochs", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Observe: Training accuracy keeps improving with more epochs.")
print("Validation accuracy may plateau or decrease — a sign of OVERFITTING.")
print(
    "The gap between train and val accuracy indicates how much the model has overfit."
)

### 4.3 Grid Search for Best Hyperparameters

Manually tuning hyperparameters is tedious. **Grid search** automates this by trying all combinations and using cross-validation to find the best one.


In [ ]:
# Define parameter grid
param_grid = {
    "hidden_layer_sizes": [(5,), (10,), (20,), (50,), (10, 5), (20, 10)],
    "learning_rate_init": [0.001, 0.01, 0.1],  # Include 0.1 based on section 4.1
    "alpha": [0.0001, 0.001, 0.01],  # L2 regularization
}

# Create base model
mlp_base = MLPClassifier(
    activation="relu", solver="adam", max_iter=500, random_state=42
)

# Grid search with cross-validation
print("Running grid search (this may take a moment)...")
grid_search = GridSearchCV(
    mlp_base,
    param_grid,
    cv=5,
    scoring="accuracy",
    return_train_score=True,
    n_jobs=-1,  # Use all CPU cores
)

# Fit on combined train+val for CV (we kept test separate)
X_trainval_scaled = np.vstack([X_train_scaled, X_val_scaled])
y_trainval = np.hstack([y_train, y_val])

grid_search.fit(X_trainval_scaled, y_trainval)

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV accuracy: {grid_search.best_score_:.3f}")

In [ ]:
# Show top 10 parameter combinations
results_df = pd.DataFrame(grid_search.cv_results_)
results_df = results_df.sort_values("mean_test_score", ascending=False)

print("Top 10 Hyperparameter Combinations:")
print("=" * 70)

top_10 = results_df[["params", "mean_test_score", "std_test_score"]].head(10)
for i, row in top_10.iterrows():
    print(
        f"CV Acc: {row['mean_test_score']:.3f} (±{row['std_test_score']:.3f})  |  {row['params']}"
    )

### 4.4 Final Evaluation on Test Set

Now we use the test set (which we've kept completely separate) to estimate real-world performance.


In [ ]:
# Get best model and evaluate on test set
best_mlp = grid_search.best_estimator_

y_pred_test = best_mlp.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"Test Accuracy: {test_accuracy:.3f}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred_test, target_names=["Class 0", "Class 1"]))

In [ ]:
# Visualize final model
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Decision boundary on test data
plot_decision_boundary(
    best_mlp,
    X_test_scaled,
    y_test,
    ax=axes[0],
    title=f"Best MLP on Test Data\nAccuracy: {test_accuracy:.3f}",
)

# Confusion matrix
ConfusionMatrixDisplay.from_estimator(
    best_mlp,
    X_test_scaled,
    y_test,
    display_labels=["Class 0", "Class 1"],
    cmap="Blues",
    ax=axes[1],
)
axes[1].set_title("Confusion Matrix")

plt.tight_layout()
plt.show()

### 4.5 Your Turn: Full Pipeline on Circles Dataset

**Task:** Apply everything you've learned to the circles dataset:

1. Split into train/val/test
2. Scale the features
3. Use grid search to find the best hyperparameters
4. Evaluate on the test set


In [ ]:
# TODO: Complete pipeline for circles dataset

# 1. Split data

# 2. Scale features

# 3. Grid search

# 4. Evaluate on test set

### 4.6 Reflection Questions

1. How does the MLP's performance compare to the SVM from Lab 09 on similar datasets?
2. What was the effect of the learning rate on training stability?
3. Did you observe overfitting? How could you tell?
4. Which hyperparameter had the biggest impact on performance?


---

## Summary

In this lab, you learned:

1. **MLPs with hidden layers** can learn non-linear decision boundaries that linear classifiers cannot.

2. **Activation functions** (sigmoid, tanh, ReLU) introduce non-linearity. ReLU is most common today because it avoids the vanishing gradient problem.

3. **Network architecture** (width and depth) controls model capacity — more neurons/layers can model more complex patterns but risk overfitting.

4. **Hyperparameters** like learning rate, number of epochs, and regularization (alpha) significantly affect training dynamics and final performance.

5. **Train/validation/test splits** are essential for proper model selection and unbiased performance estimation.

### Key Takeaways

- Always evaluate final performance on a held-out test set

- Start simple: A single hidden layer with 10-50 neurons often works well- Grid search with cross-validation automates hyperparameter tuning

- ReLU activation is a safe default choice- Use validation accuracy to detect overfitting (train >> val)
